## 1️⃣ Chargement des données

In [1]:
import pandas as pd
import json
import folium
from folium import plugins
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries loaded")

✓ Libraries loaded


In [2]:
# Charger la table avec coordonnées
df = pd.read_parquet("../../../../data/silver/silver_amenagements_with_coordinates")

print(f"✓ Loaded {len(df):,} aménagements")
print(f"✓ Colonnes: {list(df.columns)}")

✓ Loaded 4,592 aménagements
✓ Colonnes: ['amenagement_id', 'nom', 'commune1', 'insee1', 'commune2', 'insee2', 'reseau', 'financementac', 'typeamenagement', 'typeamenagement2', 'positionnement', 'senscirculation', 'environnement', 'localisation', 'typologiepiste', 'revetementpiste', 'domanialite', 'reglementation', 'zonecirculationapaisee', 'anneelivraison', 'longueur_m', 'observation', 'is_valid', 'coordiantes']


## 2️⃣ Statistiques générales

In [3]:
# Aperçu des données
df.head()

,amenagement_id,nom,commune1,insee1,commune2,insee2,reseau,financementac,typeamenagement,typeamenagement2,...,typologiepiste,revetementpiste,domanialite,reglementation,zonecirculationapaisee,anneelivraison,longueur_m,observation,is_valid,coordiantes
0,6416,Chemin des Grenettes,Quincieux,69163,None,None,Réseau de desserte,PROX/FIC DGEEP/VTP,Double sens cyclable,None,...,None,None,Métropole,Contresens (sens réservé),None,2021.0,1150.48,Arrêté n°2020/204 le 08/12/20,None,"[[[4.75558583, 45.91976734], [4.75598333, 45.9..."
1,3751,Rond-point des Monts d'Or,Lyon 9e Arrondissement,69389,None,None,Réseau de desserte,OR DIEM/STEP,Piste Cyclable,None,...,Piste sur trottoir,"Matériaux liés (asphaltes, enrobés, bétons et ...",Métropole,Vélo facultatif,None,2017.0,46.21,Arrêté N°2017RP34014 du 01/12/2017,None,"[[[4.80747525, 45.77693622], [4.80755862, 45.7..."
2,3434,Promenade Lénine,Vaulx-en-Velin,69256,None,None,Réseau secondaire,OR DIEM/STEP,Voie verte,None,...,None,None,Métropole,Voie verte,None,2017.0,54.60,None,None,"[[[4.94146652, 45.77696572], [4.94206684, 45.7..."
3,345,Piste parallèle au chemin des Vernes,Décines-Charpieu,69275,None,None,Réseau de desserte,PROX/FIC DGEEP/VTP,Piste Cyclable,None,...,None,None,Métropole,None,None,NaN,1131.86,None,None,"[[[4.9579758, 45.79410482], [4.95934142, 45.79..."
4,5870,8ème Rue Cité Berliet,Saint-Priest,69290,None,None,Réseau secondaire,PPI DGEEP VTP,Piste Cyclable,None,...,Piste sur trottoir,"Matériaux liés (asphaltes, enrobés, bétons et ...",Métropole,Vélo facultatif,Zone 30,2012.0,37.18,None,None,"[[[4.90086774, 45.70599098], [4.9008211, 45.70..."


In [5]:
# 🔍 VÉRIFICATION: Les coordonnées sont-elles complètes ou tronquées?
print("=== Vérification de la complétude des données ===\n")

# Prendre un exemple
test_row = df.iloc[0]
coords_str = test_row['coordiantes']

print(f"Aménagement: {test_row['nom']}")
print(f"Longueur de la chaîne JSON: {len(coords_str)} caractères")

# Parser pour vérifier
try:
    coords_parsed = json.loads(coords_str)
    print(f"✅ JSON valide!")
    print(f"Nombre de segments: {len(coords_parsed)}")
    total_points = sum(len(segment) for segment in coords_parsed)
    print(f"Nombre total de points: {total_points}")
    
    # Afficher les premiers et derniers points
    print(f"\n📍 Premier point: {coords_parsed[0][0]}")
    print(f"📍 Dernier point: {coords_parsed[-1][-1]}")
    
    # Vérifier que ce n'est PAS tronqué
    if coords_str.endswith('...'):
        print("\n❌ PROBLÈME: Les données sont tronquées!")
    else:
        print("\n✅ Les coordonnées sont COMPLÈTES (pas tronquées)")
        
except json.JSONDecodeError as e:
    print(f"\n❌ ERREUR: JSON invalide - {e}")
    print(f"Les données sont probablement tronquées!")
    print(f"\nDerniers 50 caractères: ...{coords_str[-50:]}")

=== Vérification de la complétude des données ===

Aménagement: Chemin des Grenettes
Longueur de la chaîne JSON: 726 caractères
✅ JSON valide!
Nombre de segments: 1
Nombre total de points: 27

📍 Premier point: [4.75558583, 45.91976734]
📍 Dernier point: [4.76679695, 45.913244]

✅ Les coordonnées sont COMPLÈTES (pas tronquées)


In [6]:
# Statistiques sur les coordonnées
print("=== Statistiques Coordonnées ===")
print(f"Aménagements avec coordonnées: {df['coordiantes'].notna().sum():,} / {len(df):,}")
print(f"Couverture: {df['coordiantes'].notna().sum() / len(df) * 100:.1f}%")

# Analyser la complexité des géométries
def count_points(coords_str):
    if pd.isna(coords_str):
        return 0
    try:
        coords = json.loads(coords_str)
        return sum(len(segment) for segment in coords)
    except:
        return 0

def count_segments(coords_str):
    if pd.isna(coords_str):
        return 0
    try:
        coords = json.loads(coords_str)
        return len(coords)
    except:
        return 0

df['n_points'] = df['coordiantes'].apply(count_points)
df['n_segments'] = df['coordiantes'].apply(count_segments)

print(f"\n=== Complexité des géométries ===")
print(f"Moyenne de points par aménagement: {df['n_points'].mean():.1f}")
print(f"Moyenne de segments par aménagement: {df['n_segments'].mean():.1f}")
print(f"Max points: {df['n_points'].max()}")
print(f"Max segments: {df['n_segments'].max()}")

=== Statistiques Coordonnées ===
Aménagements avec coordonnées: 4,592 / 4,592
Couverture: 100.0%

=== Complexité des géométries ===
Moyenne de points par aménagement: 10.6
Moyenne de segments par aménagement: 1.0
Max points: 214
Max segments: 3


In [7]:
# Distribution des types d'aménagements
print("=== Top 10 Types d'aménagements ===")
df['typeamenagement'].value_counts().head(10)

=== Top 10 Types d'aménagements ===


typeamenagement
Bande Cyclable                               1462
Double sens cyclable                         1323
Piste Cyclable                                805
Voie verte                                    534
Couloir bus vélo non élargi                   208
Chaussée à voie centrale banalisée (CVCB)     183
Couloir bus vélo élargi                        60
Goulotte ou rampe                              17
Name: count, dtype: int64

In [6]:
# Aménagements les plus complexes (plus de points)
print("=== Top 10 Aménagements les plus complexes (géométrie) ===")
df.nlargest(10, 'n_points')[['amenagement_id', 'nom', 'commune1', 'typeamenagement', 'n_points', 'n_segments', 'longueur_m']]

=== Top 10 Aménagements les plus complexes (géométrie) ===


,amenagement_id,nom,commune1,typeamenagement,n_points,n_segments,longueur_m
297,6837,Route du Mont Cindre,Saint-Cyr-au-Mont-d'Or,Chaussée à voie centrale banalisée (CVCB),214,1,1717.61
4390,2684,Promenade du Biézin,Chassieu,Voie verte,195,1,3346.69
683,755,Anneau Bleu,Décines-Charpieu,Voie verte,171,1,4089.75
464,95,Promenade Parc de Méribel,Vaulx-en-Velin,Piste Cyclable,131,1,3760.04
628,2487,Chemin de Serin à la Croix-Rousse,Lyon 4e Arrondissement,Piste Cyclable,126,1,1097.52
4312,753,Avenue de l'Hippodrome,Rillieux-la-Pape,Piste Cyclable,122,1,1796.73
4389,763,Anneau Bleu,Meyzieu,Voie verte,117,1,4763.68
477,2655,Chemin de Halage,Meyzieu,Voie verte,110,1,3454.41
3486,6648,Route des Crêtes,Saint-Cyr-au-Mont-d'Or,Chaussée à voie centrale banalisée (CVCB),100,1,1543.35
333,6388,Chemin de Maintenue,Saint-Germain-au-Mont-d'Or,Chaussée à voie centrale banalisée (CVCB),98,1,1834.81


## 3️⃣ Exemples de coordonnées

In [ ]:
# Afficher les coordonnées d'un aménagement en détail
exemple_id = df.iloc[0]['amenagement_id']
exemple = df[df['amenagement_id'] == exemple_id].iloc[0]

print(f"🚴 Aménagement: {exemple['nom']}")
print(f"   ID: {exemple['amenagement_id']}")
print(f"   Commune: {exemple['commune1']}")
print(f"   Type: {exemple['typeamenagement']}")
print(f"   Longueur: {exemple['longueur_m']:.1f} m")
print(f"   Année: {exemple['anneelivraison']}")
print(f"\n📊 Géométrie:")
print(f"   Segments: {exemple['n_segments']}")
print(f"   Points: {exemple['n_points']}")

# Afficher les coordonnées - VERSION COMPLÈTE
coords = json.loads(exemple['coordiantes'])

print(f"\n📍 Coordonnées complètes:")
print(f"   Longueur string JSON: {len(exemple['coordiantes'])} caractères")
print(f"   Nombre de segments: {len(coords)}")

# Afficher quelques segments pour vérification
print(f"\n   Segment 1 (premiers points):")
for i, point in enumerate(coords[0][:3]):  # 3 premiers points
    print(f"      Point {i+1}: lon={point[0]:.6f}, lat={point[1]:.6f}")
if len(coords[0]) > 3:
    print(f"      ... ({len(coords[0])-3} autres points)")

if len(coords) > 1:
    print(f"\n   Dernier segment (derniers points):")
    last_segment = coords[-1]
    for i, point in enumerate(last_segment[-3:]):  # 3 derniers points
        print(f"      Point {len(last_segment)-3+i+1}: lon={point[0]:.6f}, lat={point[1]:.6f}")

# Afficher le JSON complet (première fois sans troncature)
print(f"\n📄 JSON complet (aperçu):")
json_str = json.dumps(coords, indent=2, ensure_ascii=False)
if len(json_str) > 1000:
    print(f"{json_str[:500]}")
    print(f"\n... [{len(json_str)-1000} caractères omis pour l'affichage] ...\n")
    print(f"{json_str[-500:]}")
else:
    print(json_str)

🚴 Aménagement: Chemin des Grenettes
   ID: 6416
   Commune: Quincieux
   Type: Double sens cyclable
   Longueur: 1150.5 m
   Année: 2021.0

📊 Géométrie:
   Segments: 1
   Points: 27

📍 Coordonnées (JSON):
[
  [
    [
      4.75558583,
      45.91976734
    ],
    [
      4.75598333,
      45.91956566
    ],
    [
      4.75640174,
      45.91934853
    ],
    [
      4.75668772,
      45.91919662
    ],
    [
      4.75677711,
      45.91916087
    ],
    [
      4.75689136,
      45.91913229
    ],
    [
      4.75700567,
      45.91910182
    ],
    [
      4.75778301,
      45.91869925
    ],
    [
      4.75821434,
      45.91845897
    ],
    [
      4.7583727,
      45.91834934
    ],
    [...


## 4️⃣ Visualisation sur carte interactive (Folium)

In [14]:
# Créer une carte centrée sur Lyon
lyon_center = [45.7640, 4.8357]
m = folium.Map(location=lyon_center, zoom_start=12, tiles='OpenStreetMap')

# Ajouter quelques aménagements à la carte (limiter à 50 pour la performance)
sample = df.nlargest(50, 'longueur_m')  # Les 50 plus longs

print(f"Affichage de {len(sample)} aménagements sur la carte...")

# Palette de couleurs selon le type
type_colors = {
    'Piste Cyclable': 'blue',
    'Bande Cyclable': 'green',
    'Voie verte': 'darkgreen',
    'Double sens cyclable': 'orange',
    'Chaussée à voie centrale banalisée (CVCB)': 'purple'
}

for idx, row in sample.iterrows():
    if pd.isna(row['coordiantes']):
        continue
    
    try:
        coords = json.loads(row['coordiantes'])
        
        # Couleur selon le type
        color = type_colors.get(row['typeamenagement'], 'gray')
        
        # Pour chaque segment
        for segment in coords:
            # Convertir [lon, lat] en [lat, lon] pour Folium
            locations = [[lat, lon] for lon, lat in segment]
            
            # Ajouter la ligne
            folium.PolyLine(
                locations=locations,
                color=color,
                weight=3,
                opacity=0.7,
                popup=f"{row['nom']}<br>Type: {row['typeamenagement']}<br>Longueur: {row['longueur_m']:.0f}m<br>Année: {row['anneelivraison']}",
                tooltip=row['nom']
            ).add_to(m)
    except Exception as e:
        print(f"Erreur pour {row['amenagement_id']}: {e}")
        continue

# Ajouter une légende
legend_html = '''
<div style="position: fixed; 
            top: 10px; right: 10px; width: 200px; height: auto; 
            background-color: white; z-index:9999; font-size:14px;
            border:2px solid grey; border-radius: 5px; padding: 10px">
    <p style="margin:0; font-weight:bold">Types d'aménagements</p>
    <p style="margin:0"><span style="color:blue">━━</span> Piste Cyclable</p>
    <p style="margin:0"><span style="color:green">━━</span> Bande Cyclable</p>
    <p style="margin:0"><span style="color:darkgreen">━━</span> Voie verte</p>
    <p style="margin:0"><span style="color:orange">━━</span> Double sens cyclable</p>
    <p style="margin:0"><span style="color:purple">━━</span> CVCB</p>
    <p style="margin:0"><span style="color:gray">━━</span> Autres</p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

print("✓ Carte générée!")

# OPTION 1: Afficher dans VSCode (si supporté)
# Décommenter la ligne suivante pour afficher directement dans le notebook
display(m)

# OPTION 2: Sauvegarder et ouvrir dans le navigateur
import os
import webbrowser

output_html = "/exports/carte_amenagements.html"
os.makedirs("/exports", exist_ok=True)
m.save(output_html)
print(f"✓ Carte sauvegardée: {output_html}")
print(f"  → Ouvrir dans le navigateur: file://{os.path.abspath(output_html)}")

# Décommenter pour ouvrir automatiquement dans le navigateur
webbrowser.open('file://' + os.path.abspath(output_html))


Affichage de 50 aménagements sur la carte...
✓ Carte générée!


✓ Carte sauvegardée: /exports/carte_amenagements.html
  → Ouvrir dans le navigateur: file://c:\exports\carte_amenagements.html


True

## 5️⃣ Carte par commune

In [ ]:
# Statistiques par commune
stats_commune = df.groupby('commune1').agg({
    'amenagement_id': 'count',
    'longueur_m': 'sum',
    'n_points': 'sum'
}).rename(columns={
    'amenagement_id': 'n_amenagements',
    'longueur_m': 'longueur_totale_m',
    'n_points': 'total_points'
}).sort_values('longueur_totale_m', ascending=False)

print("=== Top 15 Communes (par longueur totale d'aménagements) ===")
stats_commune.head(15)

In [ ]:
# Visualiser une commune spécifique
commune_focus = 'Lyon'  # Changez ici pour une autre commune

df_commune = df[df['commune1'].str.contains(commune_focus, case=False, na=False)]
print(f"🏙️ Commune: {commune_focus}")
print(f"   Aménagements: {len(df_commune):,}")
print(f"   Longueur totale: {df_commune['longueur_m'].sum():,.0f} m ({df_commune['longueur_m'].sum()/1000:.1f} km)")

# Carte de la commune
m_commune = folium.Map(location=lyon_center, zoom_start=13, tiles='OpenStreetMap')

# Limiter à 100 aménagements pour la performance
sample_commune = df_commune.head(100)

for idx, row in sample_commune.iterrows():
    if pd.isna(row['coordiantes']):
        continue
    
    try:
        coords = json.loads(row['coordiantes'])
        color = type_colors.get(row['typeamenagement'], 'gray')
        
        for segment in coords:
            locations = [[lat, lon] for lon, lat in segment]
            folium.PolyLine(
                locations=locations,
                color=color,
                weight=3,
                opacity=0.7,
                popup=f"{row['nom']}<br>Type: {row['typeamenagement']}<br>Longueur: {row['longueur_m']:.0f}m",
                tooltip=row['nom']
            ).add_to(m_commune)
    except:
        continue

m_commune.get_root().html.add_child(folium.Element(legend_html))

print(f"✓ Carte de {commune_focus} générée (affichant {len(sample_commune)} aménagements)")

# OPTION 1: Afficher dans VSCode
display(m_commune)

# OPTION 2: Sauvegarder et ouvrir dans le navigateur
output_html_commune = f"../exports/carte_{commune_focus.lower()}.html"
m_commune.save(output_html_commune)
print(f"✓ Carte sauvegardée: {output_html_commune}")

# Décommenter pour ouvrir automatiquement
# webbrowser.open('file://' + os.path.abspath(output_html_commune))


## 6️⃣ Export d'un exemple pour analyse

In [ ]:
# Exporter un échantillon des aménagements avec leurs coordonnées
sample_export = df.head(100)[[
    'amenagement_id', 'nom', 'commune1', 'typeamenagement', 
    'longueur_m', 'anneelivraison', 'n_points', 'n_segments', 'coordiantes'
]]

output_path = "../exports/sample_amenagements_with_coords.csv"
sample_export.to_csv(output_path, index=False)
print(f"✓ Échantillon exporté: {output_path}")

## ✅ Résumé

Ce notebook permet de:
- ✅ Charger et explorer la table `silver_amenagements_with_coordinates`
- ✅ Analyser la complexité des géométries (points, segments)
- ✅ Visualiser les aménagements sur une carte interactive
- ✅ Filtrer par commune et type
- ✅ Exporter des échantillons pour analyse

**Format de la colonne `coordiantes`:**
```json
[
  [[lon1, lat1], [lon2, lat2], ...],  // Segment 1
  [[lon3, lat3], [lon4, lat4], ...]   // Segment 2
]
```

## 💡 Comment afficher les cartes

### Dans VSCode (recommandé):
1. **Exécuter les cellules normalement** - VSCode Jupyter supporte l'affichage des cartes Folium via `display(m)`
2. Si la carte ne s'affiche pas, vérifier que le notebook est **Trusted** (coin supérieur droit)

### Dans le navigateur:
1. Les cartes sont automatiquement **sauvegardées en HTML** dans `exports/`
2. Décommenter la ligne `webbrowser.open(...)` pour ouvrir automatiquement
3. Ou ouvrir manuellement les fichiers HTML:
   - `exports/carte_amenagements.html` (carte principale)
   - `exports/carte_lyon.html` (carte par commune)

### Rendre le notebook Trusted dans VSCode:
- Cliquer sur l'icône **🛡️ Not Trusted** en haut à droite
- Ou: Ctrl+Shift+P → "Notebook: Trust"
